# Bushfire Detection Model Development
Using the Edge Impulse Python SDK with TensorFlow and Keras to train model to detect bushfires vs. no bushfire within a given image.

#Install Dependencies

In [1]:
!python -m pip install tensorflow==2.12.0 edgeimpulse

# Import Packages

In [16]:
from tensorflow import keras
import edgeimpulse as ei
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import os

# Setup

In [17]:
# Settings
ei.API_KEY = "ei_c90f6349e4f2624274eff30ee840a3e6412c3135d1651ee15515404d290ce842"
labels = ['fire', 'nofire']
num_classes = len(labels)
deploy_filename = "bushfire_detection_model_cpp.zip"

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Prepare Dataset
Organise files between 'Training and Validation' and 'Testing' folders containing 'fire' and 'nofire' images.


In [19]:
# Define paths for your dataset
train_val_directory = 'drive/MyDrive/forest_fire/Training and Validation'
test_directory = 'drive/MyDrive/forest_fire/Testing'

# Create an ImageDataGenerator for training and validation
train_val_datagen = ImageDataGenerator(
    rescale=1.0 / 255,        # Normalize pixel values
    validation_split=0.2      # Use 20% of the training data for validation
)

# Create generators for training and validation data
train_generator = train_val_datagen.flow_from_directory(
    train_val_directory,
    target_size=(128, 128),   # Resize images to 128x128
    batch_size=32,             # Number of images to return in each batch
    class_mode='categorical',       # Binary classification (fire or no fire)
    subset='training',          # Set as training data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

validation_generator = train_val_datagen.flow_from_directory(
    train_val_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',       # Binary classification
    subset='validation',        # Set as validation data
    classes=['fire','nofire'] # Explicitly specify the class names
)

Found 1467 images belonging to 2 classes.
Found 365 images belonging to 2 classes.


Setup of 'Testing' directory

In [20]:
# Create an ImageDataGenerator for testing (no split needed)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create a generator for the test data
test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',       # Binary classification
    shuffle=False,             # Don't shuffle test data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

Found 68 images belonging to 2 classes.


#Building and compiling model

In [34]:
# Build your model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    # tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.GlobalAveragePooling2D(),
    # tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Testing model setup

In [35]:
print(train_generator.class_indices)

{'fire': 0, 'nofire': 1}


# Training Model

In [36]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/10
45/45 [==============================] - 24s 530ms/step - loss: 0.6311 - accuracy: 0.6774 - val_loss: 0.5735 - val_accuracy: 0.7983
Epoch 2/10
45/45 [==============================] - 24s 523ms/step - loss: 0.4818 - accuracy: 0.8571 - val_loss: 0.4821 - val_accuracy: 0.7955
Epoch 3/10
45/45 [==============================] - 24s 530ms/step - loss: 0.3855 - accuracy: 0.8641 - val_loss: 0.4326 - val_accuracy: 0.8125
Epoch 4/10
45/45 [==============================] - 24s 525ms/step - loss: 0.3395 - accuracy: 0.8683 - val_loss: 0.4022 - val_accuracy: 0.8125
Epoch 5/10
45/45 [==============================] - 24s 535ms/step - loss: 0.3161 - accuracy: 0.8764 - val_loss: 0.3951 - val_accuracy: 0.8324
Epoch 6/10
45/45 [==============================] - 24s 528ms/step - loss: 0.3001 - accuracy: 0.8836 - val_loss: 0.3919 - val_accuracy: 0.8295
Epoch 7/10
45/45 [==============================] - 26s 562ms/step - loss: 0.2852 - accuracy: 0.8913 - val_loss: 0.3850 - val_accuracy: 0.8352

#Test Accuracy and Evaluation

In [37]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Accuracy: {test_accuracy:.2f}')

2/2 [==============================] - 1s 194ms/step - loss: 0.4134 - accuracy: 0.8438
Test Accuracy: 0.84


# Model to System Compatibility

In [39]:
# List the available profile target devices
ei.model.list_profile_devices()

['alif-he',
 'alif-hp',
 'ambiq-apollo4',
 'ambiq-apollo5',
 'arduino-nano-33-ble',
 'arduino-nicla-vision',
 'arduino-nicla-vision-m4',
 'portenta-h7',
 'brainchip-akd1000',
 'brickml',
 'cortex-m4f-80mhz',
 'cortex-m7-216mhz',
 'nxp-imx93-npu',
 'espressif-esp32',
 'himax-we-i',
 'himax-wiseeye2',
 'himax-wiseeye2-ethos',
 'imdt-v2h-cpu',
 'imdt-v2h',
 'infineon-cy8ckit-062s2',
 'infineon-cy8ckit-062-ble',
 'mbp-16-2020',
 'memryx-mx3',
 'microchip-sama7g54',
 'nordic-nrf52840-dk',
 'nordic-nrf5340-dk',
 'nordic-nrf9151-dk',
 'nordic-nrf9160-dk',
 'nordic-nrf9161-dk',
 'jetson-nano',
 'jetson-orin-nx',
 'jetson-orin-nano',
 'openmv-h7p',
 'particle-boron',
 'particle-p2',
 'qualcomm-rb3-gen2-dk',
 'raspberry-pi-4',
 'raspberry-pi-5',
 'raspberry-pi-rp2040',
 'renesas-ck-ra6m5',
 'renesas-ek-ra8d1',
 'renesas-rzg2l',
 'renesas-rzv2h-cpu',
 'renesas-rzv2h',
 'renesas-rzv2l-cpu',
 'renesas-rzv2l',
 'st-iot-discovery-kit',
 'st-stm32n6',
 'seeed-sense-cap',
 'wio-terminal',
 'seeed-visio

In [45]:
#Estimate the RAM, ROM, and inference time for our model on the target hardware family
try:
    profile = ei.model.profile(model=model,
                               device='openmv-h7p')
    print(profile.summary())
except Exception as e:
    print(f"Could not profile: {e}")

Target results for float32:
{
    "variant": "float32",
    "device": "openmv-h7p",
    "tfliteFileSizeBytes": 6056,
    "isSupportedOnMcu": true,
    "memory": {
        "tflite": {
            "ram": 2677518,
            "rom": 56008,
            "arenaSize": 2677254
        },
        "eon": {
            "ram": 2230064,
            "rom": 29688,
            "arenaSize": 2229024
        },
        "eonRamOptimized": {
            "ram": 4067016,
            "rom": 38656,
            "arenaSize": 4065184
        }
    },
    "customMetrics": [],
    "hasPerformance": false
}


Performance on device types:
{
    "variant": "float32",
    "lowEndMcu": {
        "description": "Estimate for a Cortex-M0+ or similar, running at 40MHz",
        "timePerInferenceMs": 43242,
        "memory": {
            "tflite": {
                "ram": 2677441,
                "rom": 44600
            },
            "eon": {
                "ram": 2230016,
                "rom": 27352
            },
   

# Deploying and Downloading Model

In [41]:
import tensorflow as tf

# Define the output and input types for your model
model_output_type = ei.model.output_type.Classification(labels=labels)
model_input_type = ei.model.input_type.ImageInput()

# Define filenames
saved_model_dir = "saved_model"
tflite_model_filename = "model.tflite"
deploy_filename = "tflite_model_zip.zip"

# Save your model as a TensorFlow SavedModel
model.save(saved_model_dir)  # Save the model correctly

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the converted TFLite model
with open(tflite_model_filename, "wb") as f:
    f.write(tflite_model)

print(f"Model converted to TensorFlow Lite and saved as {tflite_model_filename}")

# Deploy the TFLite model as a .zip file
import zipfile

try:
    with zipfile.ZipFile(deploy_filename, 'w') as zipf:
        zipf.write(tflite_model_filename)
    print(f"Model successfully packaged as {deploy_filename}")
except Exception as e:
    print(f"Could not create zip file: {e}")


Model deployment successful.
Model exported as saved_model_zip.zip


In [42]:
from google.colab import files
import shutil

shutil.make_archive("saved_model", 'zip', "saved_model")
files.download("saved_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Model Specifications
Code to check if model has been trained correctly

In [44]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 global_average_pooling2d_2   (None, 32)               0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_7 (Dense)             (None, 2)                 66        
                                                                 
Total params: 962
Trainable params: 962
Non-trainable params: 0
_________________________________________________________________
